This notebook is going to be an adaptation of Zach Mueller's a walk with fastai. Where he creates a Bayesian optimisation on this dataset.

https://github.com/muellerzr/Practical-Deep-Learning-for-Coders-2.0/blob/master/Tabular%20Notebooks/02_Bayesian_Optimization.ipynb

installing/updating fastai

In [2]:
!pip install -Uqq fastai

lets import all the tabular functions,given by fastai

In [3]:
from fastai.tabular.all import *

Loading data into a dataframe from path. Path is a urllib object which has a lot of added functionalities than just having a text path

In [5]:
path = Path('../input/adult-census-income')

In [6]:
df = pd.read_csv(path/'adult.csv')

Installing Bayesian optimisation package

In [8]:
!pip install bayesian-optimization -q

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [9]:
from bayes_opt import BayesianOptimization

Now we create a fit_with function for training on hyperparameters and giving us the accuracy back.

In [12]:
def fit_with(lr:float, wd:float, dp:float, n_layers:float, layer_1:float, layer_2:float, layer_3:float):
    print(lr, wd, dp)
    if int(n_layers) == 2:
        layers = [int(layer_1), int(layer_2)]
    elif int(n_layers) == 3:
        layers = [int(layer_1), int(layer_2), int(layer_3)]
    else:
        layers = [int(layer_1)]
    config = tabular_config(embed_p = float(dp), ps = float(dp))
    learn = tabular_learner(dls, layers=layers, metrics=accuracy, config=config)
    
    with learn.no_bar() and learn.no_logging():
        learn.fit(5, lr=float(lr))
    
    acc = float(learn.validate()[1])
    
    return acc

In [15]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

creating a TabularPandas

In [22]:

cat_names = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education.num']
procs = [Categorify, FillMissing, Normalize]
y_names = 'income'
y_block = CategoryBlock()
splits = RandomSplitter()(range_of(df))

In [23]:
to = TabularPandas(df, procs=procs, cat_names=cat_names, cont_names=cont_names,
                   y_names=y_names, y_block=y_block, splits=splits)

what dls does is that a dataloader simply gives us data in batches.

In [24]:
dls = to.dataloaders(bs=512)

declaring our hyperparameters

In [25]:
hps = {
    'lr' : (1e-05, 1e-01),
    'wd' : (4e-4,0.4),
    'dp' : (0.01, 0.5),
    'n_layers' : (1,3),
    'layer_1' : (50, 200),
    'layer_2' : (100,1000),
    'layer_3' : (200,2000)
}

we declare our optimisers

In [27]:
optim = BayesianOptimization(
    f = fit_with,
    pbounds = hps,
    verbose = 2,
    random_state=1
)

In [28]:
%time optim.maximize(n_iter=10)

|   iter    |  target   |    dp     |  layer_1  |  layer_2  |  layer_3  |    lr     | n_layers  |    wd     |
-------------------------------------------------------------------------------------------------------------
0.014684121522803134 0.07482958046651729 0.21434078230426126


|  1        |  0.8374   |  0.2143   |  158.0    |  100.1    |  744.2    |  0.01468  |  1.185    |  0.07483  |
0.06852509784467198 0.3512957275818218 0.1793247562510934


|  2        |  0.8398   |  0.1793   |  109.5    |  584.9    |  954.6    |  0.06853  |  1.409    |  0.3513   |
0.014047289990137426 0.32037752964274446 0.02341992066698382


|  3        |  0.8418   |  0.02342  |  150.6    |  475.6    |  1.206e+0 |  0.01405  |  1.396    |  0.3204   |
0.0894617202837497 0.016006291379859792 0.4844481721025048


|  4        |  0.8318   |  0.4844   |  97.01    |  723.1    |  1.778e+0 |  0.08946  |  1.17     |  0.01601  |
0.0957893741197487 0.27687409473460917 0.09321690558663875


...

|  11       |  0.8378   |  0.3611   |  147.7    |  194.1    |  356.8    |  0.06553  |  2.663    |  0.08104  |
0.0005465734116890944 0.17802142541867974 0.46329843491833883


|  12       |  0.8216   |  0.4633   |  175.7    |  106.8    |  1.81e+03 |  0.000546 |  2.017    |  0.178    |
0.0404514572663943 0.31234441498440596 0.4911455437346324


|  13       |  0.8326   |  0.4911   |  79.71    |  936.3    |  947.6    |  0.04045  |  1.032    |  0.3123   |
0.029415674202259722 0.005868544291961274 0.31814795484259445


|  14       |  0.8398   |  0.3181   |  174.3    |  660.7    |  794.0    |  0.02942  |  1.182    |  0.005869 |
0.09277135723767846 0.23871276504022906 0.23800477408617662


|  15       |  0.8314   |  0.238    |  85.34    |  272.8    |  374.2    |  0.09277  |  1.858    |  0.2387   |
CPU times: user 3min 4s, sys: 5.76 s, total: 3min 10s
Wall time: 1min 35s


In [29]:
print(optim.max)

{'target': 0.8418304920196533, 'params': {'dp': 0.02341992066698382, 'layer_1': 150.57012652676033, 'layer_2': 475.57432213041426, 'layer_3': 1205.6416912023528, 'lr': 0.014047289990137426, 'n_layers': 1.3962029781697576, 'wd': 0.32037752964274446}}
